Annie Taylor
2.27.22

# Combine Result Rasters

### Imports

In [1]:
import arcpy
import os
import pandas as pd

### Set Parameters

In [2]:
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# The species/model to investigate
code = 'QUKE'
# codes = ['ARDO', 'CAME', 'CHPO', 'CLDO', 'COCO', 'QUKE', 'RUPA', 'RUUR', 'SACE', 'VAOV']
codes = ['QUKE']
# SAHM results folder
# results = r"D:/1_AMLT/1_SDM/SAHM/" + code + "/"

# folders to iterate over/into
folders = ['brt_1', 'glm_1', 'mars_1', 'rf_1', 'Maxent_1']
models = ['brt', 'glm', 'mars', 'rf', 'Maxent']

# get two main projections and save study area outlines in each of those projections
studyarea_wgs = r"D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/AMLT_StewardshipArea_UTM_10kmWGS84"
wgssr = arcpy.Describe(studyarea_wgs).spatialReference

studyarea_utm = r"D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/AMLT_StewardshipArea_UTM_10km"    
utmsr = arcpy.Describe(studyarea_utm).spatialReference

### Combine binary rasters into result rasters - iterate over all species at once

In [3]:
# codes = ['CAME']
# models = ['brt', 'rf', 'Maxent']

for code in codes:
    results = r"D:/1_AMLT/1_SDM/SAHM/" + code + "/"
    # create a list of all of the binary rasters
    rasters = []

    for model in models:
        raster = results + model + '_1/' + model + '_bin_map.tif'
        rasters.append(raster)

    # print(rasters)

    # initialize outras as the first raster in the list so that this resets each time I run the cell
    outras = arcpy.Raster(rasters[0])
    # iterate over the other rasters
    for raster in rasters[1:]:
        # Cast file name as raster to enable spatial analyst addition
        listras = arcpy.Raster(raster)
        # Add each raster to an output raster
        outras += listras

    # Save combined raster to file
    outwgs = results + code + '_AllBinary_WGS.tif'
    outwgsclip = results + code + '_AllBinary_WGS_Clip.tif'

    # if you don't want to overwrite files use this
    if os.path.exists(outwgs):
        print('File ' + outwgs + ' already exists')
    else:
        print('Saving to ' + outwgs)
        outras.save(outwgs)
    # if you want to overwrite the file use this
    # outras.save(outwgs)

    # Save a clipped version of it
    if os.path.exists(outwgsclip):
        print('File ' + outwgsclip + ' already exists')
    else:
        print('Saving to ' + outwgsclip)
        arcpy.management.Clip(outwgs, "-122.476320553 36.138630725 -120.367393001 37.328780964", outwgsclip, studyarea_wgs, clipping_geometry="ClippingGeometry", maintain_clipping_extent="MAINTAIN_EXTENT")

    # Save a UTM projected version of the clipped version)
    oututmclip = results + code + '_AllBinary_UTM_Clip.tif'

    if os.path.exists(oututmclip):
        print('File ' + oututmclip + ' already exists')
    else:
        insr = arcpy.Describe(outwgsclip).spatialReference
        arcpy.management.ProjectRaster(outwgsclip, oututmclip, utmsr, "NEAREST", in_coor_system=insr, geographic_transform="WGS_1984_(ITRF08)_To_NAD_1983_2011")
        print(oututmclip + ' projected')



Saving to D:/1_AMLT/1_SDM/SAHM/OldRuns/QUKEmess8000/QUKEmess8000_AllBinary_WGS.tif
Saving to D:/1_AMLT/1_SDM/SAHM/OldRuns/QUKEmess8000/QUKEmess8000_AllBinary_WGS_Clip.tif
D:/1_AMLT/1_SDM/SAHM/OldRuns/QUKEmess8000/QUKEmess8000_AllBinary_UTM_Clip.tif projected


### Threshold the agreement raster using 3+ models

In [4]:
# codes = ['ARDO', 'CAME']
for code in codes:
    results = r"D:/1_AMLT/1_SDM/SAHM/" + code + "/"

    outwgsclip = results + code + '_AllBinary_WGS_Clip.tif'
    threshold = results + code + '_Threshold_2plus.tif'

    # Save a clipped version of it
    if os.path.exists(threshold):
        print('File ' + threshold + ' already exists')
    else:
        out_raster = arcpy.sa.Reclassify(outwgsclip, "VALUE", "0 1 0;2 5 1", "DATA")
        out_raster.save(threshold)
        print('Created ' + threshold)

Created D:/1_AMLT/1_SDM/SAHM/OldRuns/QUKEmess8000/QUKEmess8000_Threshold_2plus.tif


### Add the threshold rasters to get a species count raster

In [12]:

# create a list of all of the threshold rasters
rasters = []

for code in codes:
    results = r"D:/1_AMLT/1_SDM/SAHM/" + code + "/"
    threshold = results + code + '_Threshold_3plus.tif'
    rasters.append(threshold)

print(rasters)

# initialize outras as the first raster in the list so that this resets each time I run the cell
outras = arcpy.Raster(rasters[0])
# iterate over the other rasters
for raster in rasters[1:]:
    # Cast file name as raster to enable spatial analyst addition
    listras = arcpy.Raster(raster)
    # Add each raster to an output raster
    outras += listras

allcount = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\TenSpeciesCount"

# Save a clipped version of it
if os.path.exists(allcount):
    print('File ' + allcount + ' already exists')
else:
    outras.save(allcount)

['D:/1_AMLT/1_SDM/SAHM/ARDO/ARDO_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/CAME/CAME_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/CHPO/CHPO_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/CLDO/CLDO_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/COCO/COCO_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/QUKE/QUKE_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/RUPA/RUPA_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/RUUR/RUUR_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/SACE/SACE_Threshold_3plus.tif', 'D:/1_AMLT/1_SDM/SAHM/VAOV/VAOV_Threshold_3plus.tif']


### Add Binary Rasters into one agreement raster (one at a time)

In [28]:
# create a list of all of the binary rasters
rasters = []

for model in models:
    raster = results + model + '_1/' + model + '_bin_map.tif'
    rasters.append(raster)

# print(rasters)

# initialize outras as the first raster in the list so that this resets each time I run the cell
outras = arcpy.Raster(rasters[0])
# iterate over the other rasters
for raster in rasters[1:]:
    # Cast file name as raster to enable spatial analyst addition
    listras = arcpy.Raster(raster)
    # Add each raster to an output raster
    outras += listras
print(outras)

# Save combined raster to file
outwgs = results + code + '_AllBinary_WGS.tif'
outwgsclip = results + code + '_AllBinary_WGS_Clip.tif'

# if you don't want to overwrite files use this
if os.path.exists(outwgs):
    print('File ' + outwgs + ' already exists')
else:
    print('Saving to ' + outwgs)
    outras.save(outwgs)
# if you want to overwrite the file use this
# outras.save(outwgs)

# Save a clipped version of it
if os.path.exists(outwgsclip):
    print('File ' + outwgsclip + ' already exists')
else:
    print('Saving to ' + outwgsclip)
    arcpy.management.Clip(outwgs, "-122.476320553 36.138630725 -120.367393001 37.328780964", outwgsclip, studyarea_wgs, clipping_geometry="ClippingGeometry", maintain_clipping_extent="MAINTAIN_EXTENT")

# Save a UTM projected version of the clipped version)
oututmclip = results + code + '_AllBinary_UTM_Clip.tif'

if os.path.exists(oututmclip):
    print('File ' + oututmclip + ' already exists')
else:
    insr = arcpy.Describe(outwgsclip).spatialReference
    arcpy.management.ProjectRaster(outwgsclip, oututmclip, utmsr, "NEAREST", in_coor_system=insr, geographic_transform="WGS_1984_(ITRF08)_To_NAD_1983_2011")
    print(oututm + ' projected')

C:\Users\annal\AppData\Local\Temp\ArcGISProTemp1552\xf4184088_dc1b_40b9_8735_ab0eb00b2107y0.afr
File D:/1_AMLT/1_SDM/SAHM/ARDO/ARDO_AllBinary_WGS.tif already exists
File D:/1_AMLT/1_SDM/SAHM/ARDO/ARDO_AllBinary_WGS_Clip.tif already exists
D:/1_AMLT/1_SDM/SAHM/ARDO/ARDO_AllBinary_UTM.tif projected
